In [1]:
! pip install pandas fuzzywuzzy python-levenshtein recordlinkage numpy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for python-levenshtein from https://files.pythonhosted.org/packages/72/8e/559c539e76bc0b1defec3da39a047fe151258efc9b215bf41db41e2c7922/python_Levenshtein-0.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for recordlinkage from https://files.pythonhosted.org/packages/12/fc/05c343d0b8e02c1b2f45256202a50f6970dae0bfac791c569a74c779c76d/recordlinkage-0.16-py3-none-any.whl.metadata
  Obtaining dependency information for Levenshtein==0.25.1 from https://files.pythonhosted.org/packages/47/19/4528246e25bb79fa8d4adae6640251c613f05eb310d79307d1ac53c7bf28/Levenshtein-0.25.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for jellyfish>=1 from https://files.pythonhosted.org/packages/7e/25/9fc8c0836ecddcdb0acd2abbf0ecf334d9425afbe1099b61ecdde981669c/jellyfish-1.0.3-cp311-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/98.

In [30]:
import pandas as pd
import recordlinkage
from recordlinkage.preprocessing import clean
from metaphone import doublemetaphone

# Load the data from CSV
df = pd.read_csv('./new.csv')

# Clean the data (optional step)
df['aggr_full_name'] = clean(df['aggr_full_name'])
df['full_name'] = clean(df['full_name'])

print("DataFrame:\n", df)

# Apply phonetic encoding for blocking
df['aggr_full_name_soundex'] = df['aggr_full_name'].apply(lambda x: doublemetaphone(x)[0])
df['full_name_soundex'] = df['full_name'].apply(lambda x: doublemetaphone(x)[0])

print("DataFrame with Soundex:\n", df)

# Create indexer to identify potential matches
indexer = recordlinkage.Index()
indexer.block('aggr_full_name_soundex')
candidate_links = indexer.index(df)

print("Candidate Links:\n", candidate_links)

# Compare records using Jaro-Winkler similarity
compare_cl = recordlinkage.Compare()
compare_cl.string('aggr_full_name', 'full_name', method='jarowinkler', threshold=0.75, label='name')

# Compute similarity features
features = compare_cl.compute(candidate_links, df)
print("Similarity Features:\n", features)

# Sum the similarity scores for name
features['score'] = features.sum(axis=1)
print("Features with Combined Score:\n", features)

# Define a threshold for automatic confirmation
threshold = 0.2  # Adjust based on your data and requirements
confirmed_matches = features[features['score'] >= threshold]
print("Confirmed Matches:\n", confirmed_matches)

# Reset index to ensure uniqueness
df.reset_index(drop=True, inplace=True)
confirmed_matches.reset_index(drop=True, inplace=True)

# Extract matched records
matched_records = []
for match_index in confirmed_matches.index:
    left_idx, right_idx = confirmed_matches.loc[match_index]
    matched_records.append([df.loc[left_idx, 'full_name'], df.loc[right_idx, 'aggr_full_name']])

# Create DataFrame from matched records
matched_df = pd.DataFrame(matched_records, columns=['full_name', 'aggr_full_name'])

# Display the matched DataFrame with only 'full_name' and 'aggr_full_name'
print("Matched DataFrame:\n", matched_df)


DataFrame:
                                      loc_id         loc_name  \
0      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
1      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
2      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
3      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
4      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
...                                     ...              ...   
11758  7e2a8a81-1462-11eb-89a4-42010a010003          A2B SPF   
11759  7e2a8a81-1462-11eb-89a4-42010a010003          A2B SPF   
11760  7e2a8a81-1462-11eb-89a4-42010a010003          A2B SPF   
11761  88cc107b-bc75-4d1a-9b35-9199e69d3b3d  A2B Warrenville   
11762  88cc107b-bc75-4d1a-9b35-9199e69d3b3d  A2B Warrenville   

      aggr_order_channel aggr_full_name aggr_soundex_score channel  \
0               UberEats     abhishek s               A120  Maghil   
1               UberEats     abhishek s               A120  Maghil   
2        

In [53]:
import pandas as pd
from fuzzywuzzy import fuzz
from metaphone import doublemetaphone

# Load the data from CSV
df = pd.read_csv('./new.csv')

# Clean the data (optional step)
df['aggr_full_name'] = df['aggr_full_name'].apply(lambda x: x.lower())
df['full_name'] = df['full_name'].apply(lambda x: x.lower())

print("DataFrame:\n", df)

# Apply phonetic encoding for blocking
df['aggr_full_name_soundex'] = df['aggr_full_name'].apply(lambda x: doublemetaphone(x)[0])
df['full_name_soundex'] = df['full_name'].apply(lambda x: doublemetaphone(x)[0])

print("DataFrame with Soundex:\n", df)

# Define a function to compute string similarity using fuzzywuzzy
def compute_similarity(row):
    return fuzz.partial_ratio(row['aggr_full_name'], row['full_name'])

# Compute similarity between pairs of records
df['similarity_score'] = df.apply(compute_similarity, axis=1)

# Define a threshold for potential matches
threshold = 75  # Adjust based on your data and requirements
potential_matches = df[df['similarity_score'] >= threshold]

print("Potential Matches:\n", potential_matches[['full_name', 'aggr_full_name']])


DataFrame:
                                      loc_id         loc_name  \
0      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
1      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
2      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
3      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
4      14d293a8-81c5-40b2-a208-be1c03a93fad   A2B Parsippany   
...                                     ...              ...   
11758  7e2a8a81-1462-11eb-89a4-42010a010003          A2B SPF   
11759  7e2a8a81-1462-11eb-89a4-42010a010003          A2B SPF   
11760  7e2a8a81-1462-11eb-89a4-42010a010003          A2B SPF   
11761  88cc107b-bc75-4d1a-9b35-9199e69d3b3d  A2B Warrenville   
11762  88cc107b-bc75-4d1a-9b35-9199e69d3b3d  A2B Warrenville   

      aggr_order_channel aggr_full_name aggr_soundex_score channel  \
0               UberEats    abhishek s.               A120  Maghil   
1               UberEats    abhishek s.               A120  Maghil   
2        

In [67]:
import pandas as pd
import recordlinkage

# Load the CSV file
data = pd.read_csv('./new3.csv')

# Assume you have 'Name' and 'OtherName' columns for comparison
name_column = 'full_name'
other_name_column = 'aggr_full_name'

# Indexing the dataset
indexer = recordlinkage.Index()
indexer.full()
pairs = indexer.index(data)

# Comparing records
compare = recordlinkage.Compare()

compare.string(name_column, other_name_column, method='jarowinkler', threshold=0.85)

features = compare.compute(pairs, data)

# Blocking
matches = features[features.sum(axis=1) > 0]

print(matches)


         0
12 5   1.0
15 5   1.0
43 12  1.0
   15  1.0
49 21  1.0
50 21  1.0
52 21  1.0


In [68]:
import pandas as pd

# Load the CSV file
data = pd.read_csv('./new3.csv')

# Define the matches with their similarity scores
matches = [
    (8, 7, 1.0), (9, 5, 1.0), (12, 5, 1.0), (15, 5, 1.0), (9, 15, 1.0), (21, 5, 1.0), (9, 21, 1.0), (40, 9, 1.0),
    (21, 40, 1.0), (43, 9, 1.0), (12, 43, 1.0), (15, 43, 1.0), (21, 43, 1.0), (12, 15, 1.0), (12, 21, 1.0),
    (46, 45, 1.0), (49, 21, 1.0), (50, 21, 1.0), (52, 5, 1.0), (12, 52, 1.0), (15, 52, 1.0), (21, 52, 1.0),
    (40, 52, 1.0), (43, 52, 1.0), (49, 52, 1.0), (50, 52, 1.0), (55, 5, 1.0), (12, 55, 1.0), (15, 55, 1.0),
    (21, 55, 1.0), (40, 55, 1.0), (43, 55, 1.0), (49, 55, 1.0), (50, 55, 1.0), (57, 21, 1.0), (59, 12, 1.0),
    (15, 59, 1.0), (21, 59, 1.0), (49, 59, 1.0), (50, 59, 1.0), (68, 67, 1.0), (75, 5, 1.0), (21, 75, 1.0),
    (40, 75, 1.0), (43, 75, 1.0), (79, 62, 1.0), (80, 62, 1.0), (85, 5, 1.0), (21, 85, 1.0), (40, 85, 1.0),
    (43, 85, 1.0), (87, 75, 1.0), (88, 62, 1.0), (90, 75, 1.0)
]

# Print corresponding full_name and aggr_full_name along with similarity score
for match in matches:
    index1, index2, similarity_score = match
    print("Index 1:", index1)
    print("Full Name 1:", data.loc[index1, 'full_name'])
    print("Index 2:", index2)
    print("Aggr Full Name 2:", data.loc[index2, 'aggr_full_name'])
    print("Similarity Score:", similarity_score)
    print()


Index 1: 8
Full Name 1: vignesh subbarorangapashyam
Index 2: 7
Aggr Full Name 2: vignesh v
Similarity Score: 1.0

Index 1: 9
Full Name 1: venkat subramanian
Index 2: 5
Aggr Full Name 2: venkateswarlu e
Similarity Score: 1.0

Index 1: 12
Full Name 1: venkatesh kunduru
Index 2: 5
Aggr Full Name 2: venkateswarlu e
Similarity Score: 1.0

Index 1: 15
Full Name 1: venkateswaran venkatramani
Index 2: 5
Aggr Full Name 2: venkateswarlu e
Similarity Score: 1.0

Index 1: 9
Full Name 1: venkat subramanian
Index 2: 15
Aggr Full Name 2: venkata sai teja r.
Similarity Score: 1.0

Index 1: 21
Full Name 1: venkateshvenkatexh
Index 2: 5
Aggr Full Name 2: venkateswarlu e
Similarity Score: 1.0

Index 1: 9
Full Name 1: venkat subramanian
Index 2: 21
Aggr Full Name 2: venkat r.
Similarity Score: 1.0

Index 1: 40
Full Name 1: venky dw
Index 2: 9
Aggr Full Name 2: venkat we ez t
Similarity Score: 1.0

Index 1: 21
Full Name 1: venkateshvenkatexh
Index 2: 40
Aggr Full Name 2: venkateswarlu e
Similarity Score: 1